# Traffic prediction modeling with GNN

In [3]:
'''# Only colab stuff
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  import torch
  import os
  print("PyTorch has version {}".format(torch.__version__))
  if 'IS_GRADESCOPE_ENV' not in os.environ:
    !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-geometric
    !pip install ogb

    os.chdir("./gdrive/MyDrive/MLG_cloned_repo/src/scripts")
except:
  print("Not in colab!")
  os.chdir("./src/scripts") '''

'# Only colab stuff\ntry:\n  from google.colab import drive\n  drive.mount(\'/content/gdrive\')\n  import torch\n  import os\n  print("PyTorch has version {}".format(torch.__version__))\n  if \'IS_GRADESCOPE_ENV\' not in os.environ:\n    !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html\n    !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html\n    !pip install torch-geometric\n    !pip install ogb\n\n    os.chdir("./gdrive/MyDrive/MLG_cloned_repo/src/scripts")\nexcept:\n  print("Not in colab!")\n  os.chdir("./src/scripts") '

### Load libraries

In [2]:
import json
import logging
import numpy as np
import os
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import datetime

#Custom scripts
import modeling_utils #as modeling_utils 
import data_preparation #as data_preparation

#Pytorch and PyG
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR 
from torch_geometric.nn import GATConv, GCNConv
from torch.utils.tensorboard import SummaryWriter

print(torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

#Lets start at src location
if os.path.exists("./src"):
  os.chdir("./src")
elif 'scripts' in os.getcwd():
  os.chdir("../")

1.13.1+cpu
Using cpu


In [ ]:
#%cd ..

### Constants and setting-up

In [10]:
# Constant config to use throughout
config = {
    'TRAIN_TEST_PROPORTION'             : (0.6, 0.1, 0.3),
    'BATCH_SIZE'                        : 64,
    'EPOCHS'                            : 700,
    'WEIGHT_DECAY'                      : 5e-5,
    'INITIAL_LR'                        : 1e-2,
    'DROPOUT'                           : 0.2,
    'ATTENTION_HEADS'                   : 8,
    'RESULTS_DIR'                       : './runs/'+time.strftime("%m-%dT%H-%M-%S")+'/',
    'data_with_already_filled_gaps'     : True,
    'counter_files_path'                : '../data/counters_filled_gaps/',                         # '../data/counters_temporal_data_2023-03-03T09-24-06/'
    'counters_nontemporal_aggregated'   : '../data/counters_non_temporal_aggregated_data.csv',
    'holidays_path'                     : '../data/holidays.csv',
    'USE_HOLIDAY_FEATURES'              : True,
    'USE_WEEKDAY_FEATURES'              : False,
    'N_GRAPHS'                          : 10*24,
    'F_IN'                              : 14*24,
    'F_OUT'                             : 7*24,
    'N_NODE'                            : 165,
    'target_col'                        : 'Sum',
    'use_tensorboard'                   : True,
    'USE_GAT'                           : True, # if True use GAT, else use GCN
    'USE_LSTM'                          : True, # if True use LSTM, else use GRU
    'LSTM_LAYER_SIZES'                  : [128, 32],  
    'GRU_LAYER_SIZES'                   : [512, 512, 64, 32],    
    'USE_EARLY_STOPPING'                : False,
    "MIN_ITERATIONS_EARLY_STOPPING"     : 30,
    "EARLY_STOPPING_TOLERANCE"          : 10,
    "LOG_BASELINE"                      : True, # if true outputs average rmse on computed on each batch,
    "DATA_DATE_SPLIT"                   : datetime.datetime(2022, 10, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
}

# Set logging level
logging.getLogger().setLevel(logging.INFO)

# Make a tensorboard writer
if config["use_tensorboard"]:
    writer = SummaryWriter()

NameError: name 'SummaryWriter' is not defined

### Model

In [ ]:
class ST_GNN(torch.nn.Module):
    """
    Spatio-Temporal Graph Neural Network which has options of using:
    1) Normal neighbor aggregation OR attention mechanism
    2) GRU temporal layer or LSTM
    """ 
    def __init__(self, device, in_channels, out_channels, n_nodes, heads=None, dropout=0.0):
        """
        Initialize the ST-GNN model
        :param in_channels Number of input channels
        :param out_channels Number of output channels
        :param n_nodes Number of nodes in the graph
        :param heads Number of attention heads to use in graph
        :param dropout Dropout probability on output of Graph Attention Network
        """
        # Set up params
        super(ST_GNN, self).__init__()
        self.device = device
        self.n_pred = out_channels
        self.dropout = dropout
        self.n_nodes = n_nodes
        #self.n_preds = 9 TODO is this needed?
        
        # Init spatial part
        if config['USE_GAT']:
            self.heads = heads
            self.gat = GATConv(in_channels=in_channels, out_channels=in_channels,
                    heads=heads, dropout=0, concat=False)
        else:
            self.gcn = GCNConv(in_channels=in_channels, out_channels=in_channels, dropout=0, concat=False)

        # Init temporal part
        if config['USE_LSTM']:
            self.lstms = []
            for layer_index, layer_size in enumerate(config["LSTM_LAYER_SIZES"]):
                if layer_index == 0: input_size = self.n_nodes
                else: input_size = config["LSTM_LAYER_SIZES"][layer_index - 1]

                lstm = torch.nn.LSTM(input_size=input_size, hidden_size=layer_size, num_layers=1, device = self.device)
                for name, param in lstm.named_parameters():
                    if 'bias' in name:
                        torch.nn.init.constant_(param, 0.0)
                    elif 'weight' in name:
                        torch.nn.init.xavier_uniform_(param)
                self.lstms.append(lstm)

            # fully-connected neural network
            self.linear = torch.nn.Linear(config["LSTM_LAYER_SIZES"][-1], self.n_nodes*self.n_pred)
        else:
            self.grus = []
            for layer_index, layer_size in enumerate(config["GRU_LAYER_SIZES"]):
                if layer_index == 0: input_size = self.n_nodes
                else: input_size = config["GRU_LAYER_SIZES"][layer_index - 1]

                gru = torch.nn.GRU(input_size=input_size, hidden_size=layer_size, num_layers=1, device = self.device)
                self.grus.append(gru)

            # fully-connected neural network
            self.linear = torch.nn.Linear(config["GRU_LAYER_SIZES"][-1], self.n_nodes*self.n_pred)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, data, device):
        """
        Forward pass of the ST-GNN model
        :param data Data to make a pass on
        :param device Device to operate on
        """
        x, edge_index = data.x, data.edge_index
        if self.device == 'cpu':
            x = torch.FloatTensor(x)
        else:
            x = torch.cuda.FloatTensor(x)

        if config['USE_GAT']:
            x = self.gat(x, edge_index)
        else:
            x = self.gcn(x, edge_index)
        x = F.dropout(x, self.dropout, training=self.training)


        batch_size = data.num_graphs
        n_node = int(data.num_nodes/batch_size)
        x = torch.reshape(x, (batch_size, n_node, data.num_features))
        x = torch.movedim(x, 2, 0)
        if config["USE_LSTM"]:
            for lstm in self.lstms:
                x, _ = lstm(x)
        else:
            for gru in self.grus:
                x, _ = gru(x)


        x = torch.squeeze(x[-1, :, :])
        x = self.linear(x)

        s = x.shape
        x = torch.reshape(x, (s[0], self.n_nodes, self.n_pred))
        x = torch.reshape(x, (s[0]*self.n_nodes, self.n_pred))
        return x

In [ ]:
@torch.no_grad()
def eval(model, device, dataloader, type='', dim_vars=None, save_predictions=False):
    model.eval()
    model.to(device)

    mae = 0
    rmse = 0
    baseline_rmse = 0
    mape = 0
    n = 0

    # Evaluate model on all data
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)
        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch, device)
            truth = batch.y.view(pred.shape)
            if i == 0:
                y_pred = torch.zeros(len(dataloader), pred.shape[0], pred.shape[1])
                y_truth = torch.zeros(len(dataloader), pred.shape[0], pred.shape[1])
            #truth = un_z_score(truth, dataloader.dataset.mean, dataloader.dataset.std_dev)
            #pred = un_z_score(pred, dataloader.dataset.mean, dataloader.dataset.std_dev)
            
            # reshape predictions
            y_pred[i, :pred.shape[0], :] = pred
            y_truth[i, :pred.shape[0], :] = truth

            # save y_prediction & true values for later analysis
            if save_predictions:
                modeling_utils.save_all_predictions(y_pred, y_truth, dim_vars, config['RESULTS_DIR'])

            # calculate batch average (take info only from x and take mean)
            pred_avg = torch.mean(batch.x[:,:config['F_IN']], axis=1, keepdim=True).repeat(1,config['F_OUT'])

            # calculate loss
            rmse += modeling_utils.RMSE(truth, pred)
            baseline_rmse += modeling_utils.RMSE(truth, pred_avg)
            mae += modeling_utils.MAE(truth, pred)
            mape += modeling_utils.MAPE(truth, pred)

            n += 1
    rmse, mae, mape, baseline_rmse = rmse / n, mae / n, mape / n, baseline_rmse / n

    logging.info(f'{type}, MAE: {round(int(mae),2)}, RMSE: {round(int(rmse),2)}, MAPE: {round(int(mape),2)}')

    #get the average score for each metric in each batch
    return rmse, mae, mape, baseline_rmse, y_pred, y_truth


def epoch_train(model, device, dataloader, optimizer, loss_fn, epoch):    
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

    model.train()
    for _, batch in enumerate(tqdm(dataloader, desc=f"Epoch {epoch}")):
        batch = batch.to(device)
        optimizer.zero_grad()
        y_pred = torch.squeeze(model(batch, device))
        loss = loss_fn()(y_pred.float(), torch.squeeze(batch.y).float())
        if config["use_tensorboard"]:
            writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()

        # multiplicative decay
        scheduler.step()

    return loss

## Train the model

In [ ]:
def model_train(train_dataloader, val_dataloader, config, device, save_test_results = False, test_dataloader = None, dim_vars = None):
    """
    Train the ST-GAT model. Evaluate on validation dataset as you go.
    :param train_dataloader Data loader of training dataset
    :param val_dataloader Dataloader of val dataset
    :param config configuration to use
    :param device Device to evaluate on
    """

    # Make the model. Each datapoint in the graph is 228x12: N x F (N = # nodes, F = time window)
    in_channels=config['F_IN']
    if config["USE_HOLIDAY_FEATURES"]: 
        in_channels += 7 * data_preparation.number_of_countries_in_holiday_dataset(config)
    if config["USE_WEEKDAY_FEATURES"]:
        in_channels += 1
    
    model = ST_GNN(
        device = device,
        in_channels=in_channels, 
        out_channels=config['F_OUT'], 
        n_nodes=config['N_NODE'], 
        heads=config['ATTENTION_HEADS'], 
        dropout=config['DROPOUT']
    )
    
    logging.info("Model initialized")
    optimizer = optim.Adam(model.parameters(), lr=config['INITIAL_LR'], weight_decay=config['WEIGHT_DECAY'])
    # optimizer = optim.SGD(model.parameters(), lr=config['INITIAL_LR'], weight_decay=config['WEIGHT_DECAY'])
    loss_fn = torch.nn.MSELoss
    model.to(device)

    # Early stopping variables
    n_iteration_since_loss_improvment = 0
    best_train_loss = 999999999999999999

    # For every epoch, train the model on training dataset. Evaluate model on validation dataset
    for epoch in range(config['EPOCHS']):
        loss = epoch_train(model, device, train_dataloader, optimizer, loss_fn, epoch)
        logging.info(f"Loss: {loss:.3f}")
        if epoch % 5 == 0:
            train_rmse, train_mae, train_mape, _, _, _ = eval(model, device, train_dataloader, 'Train')
            val_rmse, val_mae, val_mape, _, _, _ = eval(model, device, val_dataloader, 'Valid')
            if config["use_tensorboard"]:
                writer.add_scalar(f"MAE/train", train_mae, epoch)
                writer.add_scalar(f"RMSE/train", train_rmse, epoch)
                writer.add_scalar(f"MAPE/train", train_mape, epoch)
                writer.add_scalar(f"MAE/val", val_mae, epoch)
                writer.add_scalar(f"RMSE/val", val_rmse, epoch)
                writer.add_scalar(f"MAPE/val", val_mape, epoch)
        
        if config["USE_EARLY_STOPPING"]:
          if loss < best_train_loss:
            best_train_loss = loss
            n_iteration_since_loss_improvment = 0
          else: n_iteration_since_loss_improvment += 1

          if epoch >= config["MIN_ITERATIONS_EARLY_STOPPING"] and \
                n_iteration_since_loss_improvment >= config["EARLY_STOPPING_TOLERANCE"]:
            break
    logging.info("All epochs done, finished training")

    if config["use_tensorboard"]:
        writer.flush()
    # Save the model
    os.mkdir(config["RESULTS_DIR"])
    torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": loss},
            os.path.join(config["RESULTS_DIR"], "model.pt")
    )
    
    with open(os.path.join(config["RESULTS_DIR"], "config.json"), "w") as fp:
        json.dump(config, fp)

    if save_test_results:
        test_rmse, test_mae, test_mape, baseline_rmse, _, _ = eval(model, device, test_dataloader, 'Test', dim_vars, save_predictions=True)
        logging.info(f"Test RMSE:{test_rmse}")
        if config['LOG_BASELINE']:
          logging.info(f"Test BASELINE RMSE:{baseline_rmse}")
        results = {'MAE': test_mae.item(),
                    'RMSE': test_rmse.item(),
                    'MAPE': test_mape.item()}
        with open(os.path.join(config["RESULTS_DIR"], "results.json"), "w") as fp:
            json.dump(results, fp)
    return model

### Start training

In [45]:
import importlib
importlib.reload(data_preparation)
importlib.reload(modeling_utils)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\markoi\Anaconda3\envs\dars-marko\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\markoi\AppData\Local\Temp\ipykernel_21112\553531720.py", line 2, in <module>
    importlib.reload(data_preparation)
  File "c:\Users\markoi\Anaconda3\envs\dars-marko\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\markoi\Desktop\TrafficPrediction_MLG_Project\src\scripts\data_preparation.py", line 10, in <module>
    from sklearn.preprocessing import MinMaxScaler
  File "c:\Users\markoi\Anaconda3\envs\dars-marko\lib\site-packages\sklearn\__init__.py", line 82, in <module>
    from .base import clone


In [36]:
# Make runs directory if it does not exist
if not os.path.exists(config['RESULTS_DIR'].rsplit('/', 2)[0]):
    os.mkdir(config['RESULTS_DIR'].rsplit('/', 2)[0])

dataset, dim_vars = data_preparation.prepare_pyg_dataset(config)
train_g, val_g, test_g, train_vars, val_vars, test_vars = data_preparation.split_dataset(dataset, config, dim_vars = dim_vars)

# Split the Data instances in 
train_dataloader = DataLoader(train_g, batch_size=config['BATCH_SIZE'], shuffle=False)
val_dataloader = DataLoader(val_g, batch_size=config['BATCH_SIZE'], shuffle=False)
test_dataloader = DataLoader(test_g, batch_size=config['BATCH_SIZE'], shuffle=False)

INFO:root:Preparing data...
INFO:root:Holiday features successfully prepared
INFO:root:Historical counter data successfully read


KeyboardInterrupt: 

In [17]:
# Configure and train model
model = model_train(train_dataloader, val_dataloader, config, device, True, test_dataloader, val_vars)

INFO:root:Model initialized
Epoch 0: 100%|██████████| 3/3 [00:22<00:00,  7.48s/it]
INFO:root:Loss: 787492.438


KeyboardInterrupt: 